In [37]:
#ライブラリをインポート
import os #OSに依存する様々な機能を利用するためのモジュール(ファイルやディレクトリ操作など)
import re #正規表現を利用するためのモジュール
import csv  #csvファイルを扱うためのモジュール
import math #数学的計算のためのモジュール
import cv2 #画像処理のためのモジュール
import matplotlib.pyplot as plt #グラフ描画のためのモジュール
import numpy as np  #多次元配列計算のためのモジュール
import pandas as pd #データフレームを扱うためのモジュール
from scipy import signal  #信号処理のためのモジュール
from scipy.stats import skew, kurtosis  #歪度と尖度を調べるためのモジュール
from sklearn.model_selection import train_test_split  #データをトレーニング用とテスト用に分けるためのモジュール
from sklearn import preprocessing #データを正規化するためのモジュール
from sklearn.preprocessing import StandardScaler  #データを標準化するためのモジュール
from sklearn.preprocessing import LabelEncoder  #カテゴリ変数を数値化するためのモジュール
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error, r2_score, f1_score, fbeta_score#機械学習モデルの性能評価のためのモジュール
import tensorflow as tf #TensorFlow(Googleが開発したオープンソースの機械学習フレームワーク)
from tensorflow import keras  #TensorFlow用のニューラルネットワークライブラリAPI
from tensorflow.keras import layers #ニューラルネットワークのレイヤーを定義するためのモジュール
from keras.utils import np_utils

In [38]:
#メモ
#画像のサイズは縦：横=1088:1920=17:30

#よく使うgit command
#google colabと揃える
#git fetch origin main
#git reset --hard origin/main

In [39]:
#自分のGoogle Driveとドッキング
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
#GPUを使うための処理
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [41]:
#マクロの定義
N = 1 #Encoderの層の数
NUM_HEADS = 8 #アテンションヘッドの数
KEY_DIM = 30  #キー次元
DROPOUT = 0.1 #ドロップアウト率
LENGTH = 240  #リサイズ後の画像の横の長さ
HEIGHT = 136  #リサイズ後の画像の縦の長さ

In [42]:
#正解データへのパスとファイル名取得
CollectPath = "/content/drive/MyDrive/DICOMO/正解データ/"
CollectFilename = os.listdir(CollectPath)

In [43]:
#不正解データへのパスとファイル名取得
IncollectPath = "/content/drive/MyDrive/DICOMO/不正解データ/"
IncollectFilename = os.listdir(IncollectPath)

In [44]:
#教師データを格納するリストを定義
img = []

In [45]:
#正解データをグレースケールに変換・リサイズ
for i in CollectFilename:
     image = cv2.imread(CollectPath+i)
     image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
     image = cv2.resize(image, dsize=(LENGTH, HEIGHT))
     img.append(image)

In [46]:
#不正解データをグレースケールに変換・リサイズ
for i in IncollectFilename:
     image = cv2.imread(IncollectPath+i)
     image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
     image = cv2.resize(image, dsize=(LENGTH, HEIGHT))
     img.append(image)

#ndarray型に変換
img = np.array(img)

In [47]:
#教師ラベルを作成
#渡れる時を1、渡れない時を0とする
CollectLabel = np.ones(len(CollectFilename))
IncollectLabel = np.zeros(len(IncollectFilename))
label = np.concatenate((CollectLabel, IncollectLabel))

#one-hot-encoding([1, 0]や[0, 1]のようにする)
label = np_utils.to_categorical(label)

In [50]:
#トレーニングデータとテストデータの分割
x_train, x_test, y_train, y_test = train_test_split(img, label, train_size = 0.8, shuffle = True)

In [51]:
# Define the input shape
input_shape = (HEIGHT, LENGTH)
output_shape = (1,)

#形を定義(このモジュールは行列でないとダメっぽい)
x_encoder = layers.Input(shape=input_shape)

In [52]:
#Transformer Encoder Layer(BERT)
for i in range(N):
      #Multi-Head-Attention Layer
      attention_encoder = layers.MultiHeadAttention(num_heads=NUM_HEADS, key_dim=KEY_DIM, use_bias=True)(x_encoder, x_encoder, x_encoder)

      #Dropout Layer
      attention_encoder = layers.Dropout(rate=DROPOUT)(attention_encoder)
      #Add & Norm Layer
      attention_encoder = layers.LayerNormalization()(x_encoder + attention_encoder)

      #Feed-Forward-Network
      ffn_encoder = layers.Dense(LENGTH * 4, use_bias=True, activation="relu")(attention_encoder)
      ffn_encoder = layers.Dense(LENGTH, use_bias=True)(ffn_encoder)

      #Dropout Layer
      ffn_encoder = layers.Dropout(rate=DROPOUT)(ffn_encoder)
      #Add & Norm Layer
      x_encoder = layers.LayerNormalization()(attention_encoder + ffn_encoder)

In [53]:
x = layers.Flatten()(x_encoder)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(2, activation="softmax")(x)

In [54]:
model = keras.Model(inputs=x_encoder, outputs=outputs)

In [55]:
# compile model
model.compile(
    loss="categorical_crossentropy",   # 誤差関数
    optimizer="adam",     # 最適化手法
    metrics=['accuracy']
    )

In [56]:
model.fit(x_train, y_train, batch_size=32, epochs=50, shuffle=True, validation_split=0.2)

Epoch 1/50
13/13 [==============================] - 1s 18ms/step - loss: 1777.7792 - accuracy: 0.5426 - val_loss: 444.3787 - val_accuracy: 0.6598
Epoch 2/50
13/13 [==============================] - 0s 6ms/step - loss: 241.3498 - accuracy: 0.6408 - val_loss: 307.7412 - val_accuracy: 0.7216
Epoch 3/50
13/13 [==============================] - 0s 6ms/step - loss: 102.0896 - accuracy: 0.7778 - val_loss: 152.9412 - val_accuracy: 0.7526
Epoch 4/50
13/13 [==============================] - 0s 6ms/step - loss: 127.1032 - accuracy: 0.7339 - val_loss: 246.7272 - val_accuracy: 0.5979
Epoch 5/50
13/13 [==============================] - 0s 6ms/step - loss: 157.1029 - accuracy: 0.7442 - val_loss: 493.7844 - val_accuracy: 0.5773
Epoch 6/50
13/13 [==============================] - 0s 6ms/step - loss: 181.0880 - accuracy: 0.7649 - val_loss: 147.0831 - val_accuracy: 0.7629
Epoch 7/50
13/13 [==============================] - 0s 10ms/step - loss: 77.6918 - accuracy: 0.8527 - val_loss: 170.9403 - val_accurac

In [57]:
y_pred = model.predict(x_test)

4/4 [==============================] - 0s 3ms/step


In [58]:
#0.5以上の時に1, 以下の時に0とする
threshold = 0.5
y_pred = np.where(y_pred >= threshold, 1, 0)

In [59]:
accuracy_score(y_test, y_pred)

0.860655737704918

In [60]:
f1_score(y_test, y_pred, average='micro')

0.860655737704918